In [28]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [59]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def download_zip(url, output_path):
    response = requests.get(url, headers=headers, stream=True)
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"ZIP file downloaded to {output_path}")
    else:
        print(f"Failed to download. HTTP Response Code: {response.status_code}")

url = "http://www.manythings.org/anki/fra-eng.zip"
output_path = "fra-eng.zip"
download_zip(url, output_path)

path = os.getcwd()
zipfilename = os.path.join(path, output_path)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

ZIP file downloaded to fra-eng.zip


In [60]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))

전체 샘플의 개수 : 227815


In [61]:
lines

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !
...,...,...
227810,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
227811,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
227812,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...
227813,It may be impossible to get a completely error...,Il est peut-être impossible d'obtenir un Corpu...


In [62]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 6만개만 저장
lines.sample(10)

,src,tar
45024,Were you born there?,Êtes-vous née là-bas ?
33706,I lost my car keys.,J'ai perdu mes clés de voiture.
47310,He didn't say a word.,Il n'a pas dit un mot.
16101,I'm extroverted.,Je suis extraverti.
50854,Please telephone him.,Veuillez lui téléphoner.
50022,Is Tom going to jump?,Est-ce que Tom va sauter ?
4134,This is bad.,C'est mauvais.
17785,Tom is demented.,Tom est fou.
40590,I got up about five.,Je me suis levé vers cinq heures.
28012,It was inevitable.,C'était inévitable.


In [63]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
15159,I ate some cake.,\t Je mangeais du gâteau. \n
36809,Tom is an imposter.,\t Tom est un imposteur. \n
9301,They said yes.,\t Ils ont dit oui. \n
27729,I'm on page three.,\t Je suis à la page trois. \n
2657,Tom's home.,\t Tom est à la maison. \n
15011,He's dead drunk.,\t Il est ivre mort. \n
4543,You must go.,\t Tu dois partir. \n
40487,I don't like losing.,\t Je n'apprécie pas de perdre. \n
54929,Do you know who it is?,\t Savez-vous qui c'est ? \n
32977,How does this work?,\t Comment ça marche ? \n


In [64]:
# 문자 집합 구축
src_vocab = set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        src_vocab.add(char)

In [65]:
src_vocab

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é',
 'ï',
 '’',
 '€'}

In [66]:
tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [67]:
tar_vocab

{'\t',
 '\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'Ô',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '‘',
 '’',
 '\u202f',
 '‽'}

In [68]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
# 인덱스 1부터 시작하기 위함
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

source 문장의 char 집합 : 80
target 문장의 char 집합 : 104


In [69]:
src_vocab = sorted(list(src_vocab))
src_vocab

[' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é',
 'ï',
 '’',
 '€']

In [70]:
tar_vocab = sorted(list(tar_vocab))
tar_vocab

['\t',
 '\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'Ô',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '‘',
 '’',
 '\u202f',
 '‽']

In [71]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
# 키 밸류 값 바꾸기
print(src_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, 'ï': 77, '’': 78, '€': 79}


In [72]:
print(tar_to_index)

{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 35, 'J': 36, 'K': 37, 'L': 38, 'M': 39, 'N': 40, 'O': 41, 'P': 42, 'Q': 43, 'R': 44, 'S': 45, 'T': 46, 'U': 47, 'V': 48, 'W': 49, 'X': 50, 'Y': 51, 'a': 52, 'b': 53, 'c': 54, 'd': 55, 'e': 56, 'f': 57, 'g': 58, 'h': 59, 'i': 60, 'j': 61, 'k': 62, 'l': 63, 'm': 64, 'n': 65, 'o': 66, 'p': 67, 'q': 68, 'r': 69, 's': 70, 't': 71, 'u': 72, 'v': 73, 'w': 74, 'x': 75, 'y': 76, 'z': 77, '\xa0': 78, '«': 79, '»': 80, 'À': 81, 'Ç': 82, 'É': 83, 'Ê': 84, 'Ô': 85, 'à': 86, 'â': 87, 'ç': 88, 'è': 89, 'é': 90, 'ê': 91, 'ë': 92, 'î': 93, 'ï': 94, 'ô': 95, 'ù': 96, 'û': 97, 'œ': 98, '\u2009': 99, '‘': 100, '’': 101, '\u202f': 102, '‽': 103}


In [73]:
import warnings
warnings.filterwarnings('ignore')

In [74]:
encoder_input = []
for line in lines.src:
    encoded_line = []
    for c in line:
        encoded_line.append(src_to_index[c])
    encoder_input.append(encoded_line)
encoder_input

[[30, 64, 10],
 [30, 64, 10],
 [30, 64, 10],
 [30, 64, 10],
 [31, 58, 10],
 [31, 58, 10],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [46, 57, 64, 23],
 [46, 64, 72, 2],
 [46, 64, 72, 2],
 [46, 64, 72, 2],
 [27, 70, 52, 60, 2],
 [27, 70, 52, 60, 2],
 [27, 70, 52, 60, 2],
 [29, 58, 67, 54, 2],
 [31, 54, 61, 65, 2],
 [31, 58, 53, 54, 10],
 [31, 58, 53, 54, 10],
 [33, 70, 62, 65, 2],
 [33, 70, 62, 65, 10],
 [42, 69, 64, 65, 2],
 [42, 69, 64, 65, 2],
 [42, 69, 64, 65, 2],
 [46, 50, 58, 69, 2],
 [46, 50, 58, 69, 2],
 [46, 50, 58, 69, 2],
 [46, 50, 58, 69, 10],
 [46, 50, 58, 69, 10],
 [46, 50, 58, 69, 10],
 [46, 50, 58, 69, 10],
 [25, 54, 56, 58, 63, 10],
 [25, 54, 56, 58, 63, 10],
 [30, 64, 1, 64, 63, 10],
 [30, 64, 1, 64, 63, 10],
 [30, 64, 

In [75]:
decoder_input = []
for line in lines.tar:
    encoded_line = []
    for char in line:
        encoded_line.append(tar_to_index[char])
        decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])

target 문장의 정수 인코딩 : [[1, 3, 48, 52, 3, 4, 3, 2], [1, 3, 48, 52, 3, 4, 3, 2], [1, 3, 48, 52, 3, 4, 3, 2], [1, 3, 48, 52, 3, 4, 3, 2], [1, 3, 48, 52, 3, 4, 3, 2]]


In [76]:
decoder_target = []
for line in lines.tar:
    timestep = 0
    encoded_line = []
    for char in line:
        if timestep > 0:
            encoded_line.append(tar_to_index[char])
        timestep = timestep + 1
        decoder_target.append(encoded_line)
# 들여쓰기('\t') 제거
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 48, 52, 3, 4, 3, 2], [3, 48, 52, 3, 4, 3, 2], [3, 48, 52, 3, 4, 3, 2], [3, 48, 52, 3, 4, 3, 2], [3, 48, 52, 3, 4, 3, 2]]


In [77]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

source 문장의 최대 길이 : 22
target 문장의 최대 길이 : 76


In [78]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [79]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

MemoryError: Unable to allocate 46.9 GiB for an array with shape (120976952, 104) and data type float32

In [80]:
encoder_input.shape # (60000, 22, 80)
decoder_input.shape # (60000, 76, 104)
decoder_target.shape # (60000, 76, 104)

(1591802, 76)

# Seq2seq 인코더 설계

In [81]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [82]:
encoder_inputs = Input(shape=(None, src_vocab_size))

In [83]:
encoder_lstm = LSTM(units=256,return_state=True)

In [84]:
encoder_outputs , state_h , state_c = encoder_lstm(encoder_inputs)
# state_h(은닉상태hidden) , state_c(셀상태)

In [85]:
encoder_states = [state_h, state_c] # encoder_states : Context Vector

# Seq2seq 디코더 설계

In [86]:
decoder_inputs = Input(shape=(None,tar_vocab_size))

In [87]:
decoder_lstm = LSTM(units=256 , return_state=True, return_sequences=True)

In [88]:
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더는 은닉상태, 셀상태 정보가 필요없음 (훈련과정에서는)

In [89]:
decoder_softmax_layer = Dense(tar_vocab_size, activation = 'softmax')

In [90]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [91]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs) # (입력,출력)

In [92]:
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy')

In [93]:
model.fit(x = [encoder_input,decoder_input], y = decoder_target, batch_size = 64, epochs = 40, 
         validation_split=0.2)

Epoch 1/40


ValueError: in user code:

    File "C:\Users\user\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\user\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\user\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\user\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1126, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\user\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\user\anaconda3\Lib\site-packages\keras\src\engine\input_spec.py", line 235, in assert_input_compatibility
        raise ValueError(

    Va

* 테스트 과정
1. 번역 대상 입력 문장이 인코더에 들어감 -> 은닉/셀 상태 -> 디코더
2. 디코더의 입력 시그널이 전달(sos,\t)
3. 디코더는 다음 문자 예측

In [ ]:
# 테스트 과정에서의 디코더 정의

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [ ]:
# 테스트 과정에서의 인코더 정의

encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [ ]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""
    # stop_condition이 True가 될 때까지 루프 반복

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""
    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char
        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or len(decoded_sentence) > max_tar_len):
            stop_condition = True
            # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
            target_seq = np.zeros((1, 1, tar_vocab_size))
            target_seq[0, 0, sampled_token_index] = 1.
            # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
            states_value = [h, c]
    return decoded_sentence

In [ ]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index:seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력